# ETL Project | What’s Trending Across North America | Youtube Statistics

### DEPENDENCIES & SETUP

In [1]:
# Dependencies and Setup
import pandas as pd
from sqlalchemy import create_engine
import datetime
import numpy as np
import psycopg2
from pandas.io.json import json_normalize

### EXTRACT (from CSV)

In [2]:
#Read Canada data from csv
csv_file1 = "./Resources/CAvideos.csv"
videos_ca = pd.read_csv(csv_file1)

In [3]:
#Add country columns
videos_ca.loc[:,'country_id'] = 'CA'
videos_ca.loc[:,'country'] = 'Canada'

In [4]:
#retain required columns
videos_ca_subset = videos_ca[["country_id","country","video_id","category_id","trending_date","title","publish_time","likes","dislikes","views",
                              "comment_count","comments_disabled","ratings_disabled","description"]]

In [5]:
#Read Mexico data from CSV
csv_file2 = "./Resources/MXvideos.csv"
videos_mx = pd.read_csv(csv_file2, encoding='ISO-8859-1')

In [6]:
#Add country columns
videos_mx.loc[:,'country_id'] = 'MX'
videos_mx.loc[:,'country'] = 'Mexico'

In [7]:
#retain required columns
videos_mx_subset = videos_mx[["country_id","country","video_id","category_id","trending_date","title","publish_time","likes","dislikes","views",
                              "comment_count","comments_disabled","ratings_disabled","description"]]

In [8]:
#Read US data from CSV
csv_file3 = "./Resources/USvideos.csv"
videos_us = pd.read_csv(csv_file3)

In [9]:
#add country columns
videos_us.loc[:,'country_id'] = 'US'
videos_us.loc[:,'country'] = 'United States'

In [10]:
#retain required columns
videos_us_subset = videos_us[["country_id","country","video_id","category_id","trending_date","title","publish_time","likes","dislikes","views",
                              "comment_count","comments_disabled","ratings_disabled","description"]]

### EXTRACT (from JSON)

In [11]:
#dependencies
import json
import os

from pandas.io.json import json_normalize #package for flattening json in pandas df

In [12]:
# Load JSON for US categories
filepath = os.path.join("./Resources", "US_category_id.json")
with open(filepath) as jsonfile:
    us_video_json = json.load(jsonfile)

In [13]:
#Extract category id
us_video_json['items'][0]['id']

'1'

In [14]:
#Extract category name
us_video_json['items'][0]['snippet']['title']

'Film & Animation'

In [15]:
#flattening the nested json to dataframe
us_video_json_normalized = json_normalize(us_video_json['items'])

#rename column names (id to category_id) and (snippet.title to category name)
us_video_json_normalized.rename(columns={'id':'category_id',
                          'snippet.title':'category_name'}, 
                 inplace=True)
us_video_json_normalized.head()

,etag,category_id,kind,snippet.assignable,snippet.channelId,category_name
0,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,youtube#videoCategory,True,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation
1,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,youtube#videoCategory,True,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles
2,"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...",10,youtube#videoCategory,True,UCBR8-60-B28hp2BmDPdntcQ,Music
3,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...",15,youtube#videoCategory,True,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals
4,"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...",17,youtube#videoCategory,True,UCBR8-60-B28hp2BmDPdntcQ,Sports


In [16]:
#dataframe with video categories for category table
category = us_video_json_normalized[["category_id","category_name"]]
category.head(5)

,category_id,category_name
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


### TRANSFORM

In [17]:
#concatenate all the three CSV extracted datasets (US+CA+MX) to one dataset for North America
files = [videos_us_subset, videos_ca_subset, videos_mx_subset]
videos_northamerica = pd.concat(files)

In [18]:
#format dates to standard form
videos_northamerica['trending_date_formatted'] = pd.to_datetime(videos_northamerica['trending_date'],format='%y.%d.%m')
videos_northamerica['publish_date_formatted']  = videos_northamerica['publish_time'].str[:10]

In [19]:
#remove records with no video_id
videos_northamerica = videos_northamerica[videos_northamerica.video_id != "#NAME?"]

In [20]:
#remove duplicates to retain unique video ids from the north america dataset
videos_northamerica_unique = videos_northamerica.drop_duplicates('video_id',keep='first')

#retain only the required columns for table1 ' videos'
videos = videos_northamerica_unique[["video_id","title","publish_date_formatted"]]


##### Table 1  | videos (source: csv)

In [21]:
videos.head()

,video_id,title,publish_date_formatted
0,2kyS6SvSYSE,WE WANT TO TALK ABOUT OUR MARRIAGE,2017-11-13
1,1ZAPwfrtAFY,The Trump Presidency: Last Week Tonight with J...,2017-11-13
2,5qpjK5DgCt4,"Racist Superman | Rudy Mancuso, King Bach & Le...",2017-11-12
3,puqaWrEC7tY,Nickelback Lyrics: Real or Fake?,2017-11-13
4,d380meD0W0M,I Dare You: GOING BALD!?,2017-11-12


##### Table 2 | country (source: csv)

In [22]:
country = videos_northamerica.drop_duplicates('country_id',keep='first')
country = country[["country_id","country"]]
country.head()

,country_id,country
0,US,United States
0,CA,Canada
0,MX,Mexico


##### Table 3 | popularity (source: csv)

In [23]:
popularity = videos_northamerica[["country_id","video_id","category_id","trending_date_formatted","likes","dislikes","views",
                                  "comment_count","comments_disabled","ratings_disabled","description"]]
#popularity.loc[:,"id"] = np.arange(len(popularity))


popularity.insert(0, 'id', range(1, 1 + len(popularity)))
popularity.head(5)

,id,country_id,video_id,category_id,trending_date_formatted,likes,dislikes,views,comment_count,comments_disabled,ratings_disabled,description
0,1,US,2kyS6SvSYSE,22,2017-11-14,57527,2966,748374,15954,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,2,US,1ZAPwfrtAFY,24,2017-11-14,97185,6146,2418783,12703,False,False,"One year after the presidential election, John..."
2,3,US,5qpjK5DgCt4,23,2017-11-14,146033,5339,3191434,8181,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,4,US,puqaWrEC7tY,24,2017-11-14,10172,666,343168,2146,False,False,Today we find out if Link is a Nickelback amat...
4,5,US,d380meD0W0M,24,2017-11-14,132235,1989,2095731,17518,False,False,I know it's been a while since we did this sho...


##### Table 4 | category (source: json)

In [24]:
#flattening the nested json to dataframe
us_video_json_normalized = json_normalize(us_video_json['items'])

#rename column names (id to category_id) and (snippet.title to category name)
us_video_json_normalized.rename(columns={'id':'category_id',
                          'snippet.title':'category_name'}, 
                 inplace=True)
us_video_json_normalized.head()

,etag,category_id,kind,snippet.assignable,snippet.channelId,category_name
0,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,youtube#videoCategory,True,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation
1,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,youtube#videoCategory,True,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles
2,"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...",10,youtube#videoCategory,True,UCBR8-60-B28hp2BmDPdntcQ,Music
3,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...",15,youtube#videoCategory,True,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals
4,"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...",17,youtube#videoCategory,True,UCBR8-60-B28hp2BmDPdntcQ,Sports


In [25]:
#dataframe with video categories for category table
category = us_video_json_normalized[["category_id","category_name"]]
category.head()

,category_id,category_name
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


### LOAD

###### Load the dataframes as tables into relational postgres database

In [31]:
#dependencies
!pip install psycopg2

In [32]:
#connect to local database
rds_connection_string = "youknowWhat2do"

engine = create_engine(f'postgresql://{rds_connection_string}')

In [33]:
#check tables already created using the table schema sql generated from Quick database diagram
engine.table_names()

['country', 'popularity', 'videos', 'category']

##### Load Table 1 | video


In [34]:
#Use pandas to load csv converted DataFrame into database
videos.to_sql(name='videos', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "pk_videos"
DETAIL:  Key (video_id)=(2kyS6SvSYSE) already exists.

[SQL: INSERT INTO videos (video_id, title, publish_date_formatted) VALUES (%(video_id)s, %(title)s, %(publish_date_formatted)s)]
[parameters: ({'video_id': '2kyS6SvSYSE', 'title': 'WE WANT TO TALK ABOUT OUR MARRIAGE', 'publish_date_formatted': '2017-11-13'}, {'video_id': '1ZAPwfrtAFY', 'title': 'The Trump Presidency: Last Week Tonight with John Oliver (HBO)', 'publish_date_formatted': '2017-11-13'}, {'video_id': '5qpjK5DgCt4', 'title': 'Racist Superman | Rudy Mancuso, King Bach & Lele Pons', 'publish_date_formatted': '2017-11-12'}, {'video_id': 'puqaWrEC7tY', 'title': 'Nickelback Lyrics: Real or Fake?', 'publish_date_formatted': '2017-11-13'}, {'video_id': 'd380meD0W0M', 'title': 'I Dare You: GOING BALD!?', 'publish_date_formatted': '2017-11-12'}, {'video_id': 'gHZ1Qz0KiKM', 'title': '2 Weeks with iPhone X', 'publish_date_fo

##### Load Table 2 | country

In [ ]:
#Use pandas to load csv converted DataFrame into database
country.to_sql(name='country', con=engine, if_exists='append', index=False)

##### Load Table 4 | category
Note: load Table 4 before Table 3 due to dependency (foreign key category_id)

In [ ]:
category.to_sql(name='category', con=engine, if_exists='append', index=False)

##### Load Table 3 | popularity

In [ ]:
#Use pandas to load csv converted DataFrame into database
popularity.to_sql(name='popularity', con=engine, if_exists='append', index=False)

### VERIFICATION
    #Confirm data is loaded by querying the tables.

###### Querying Table 1 | videos

In [35]:
pd.read_sql_query('select * from videos', con=engine).head()

,video_id,title,publish_date_formatted
0,2kyS6SvSYSE,WE WANT TO TALK ABOUT OUR MARRIAGE,2017-11-13
1,1ZAPwfrtAFY,The Trump Presidency: Last Week Tonight with J...,2017-11-13
2,5qpjK5DgCt4,"Racist Superman | Rudy Mancuso, King Bach & Le...",2017-11-12
3,puqaWrEC7tY,Nickelback Lyrics: Real or Fake?,2017-11-13
4,d380meD0W0M,I Dare You: GOING BALD!?,2017-11-12


##### Querying Table 2 | country

In [36]:
pd.read_sql_query('select * from country', con=engine).head()

,country_id,country
0,US,United States
1,CA,Canada
2,MX,Mexico


##### Querying Table 3 | popularity

In [37]:
pd.read_sql_query('select * from popularity', con=engine).head()

,id,country_id,video_id,trending_date_formatted,category_id,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,description
0,1,US,2kyS6SvSYSE,2017-11-14,22,748374,57527,2966,15954,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,2,US,1ZAPwfrtAFY,2017-11-14,24,2418783,97185,6146,12703,False,False,"One year after the presidential election, John..."
2,3,US,5qpjK5DgCt4,2017-11-14,23,3191434,146033,5339,8181,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,4,US,puqaWrEC7tY,2017-11-14,24,343168,10172,666,2146,False,False,Today we find out if Link is a Nickelback amat...
4,5,US,d380meD0W0M,2017-11-14,24,2095731,132235,1989,17518,False,False,I know it's been a while since we did this sho...


##### Querying Table 4 | category

In [38]:
pd.read_sql_query('select * from category', con=engine).head()

,category_id,category_name
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


##### Querying all the tables using joins

In [40]:
pd.read_sql_query("""SELECT v.video_id, v.title, v.publish_date_formatted as "published date", p.trending_date_formatted as "trending date", p.views, p.likes, c.country, ca.category_id, ca.category_name
FROM videos v
JOIN popularity p
ON v.video_id = p.video_id
JOIN country c
ON p.country_id = c.country_id
JOIN category ca
ON p.category_id = ca.category_id
WHERE c.country= 'Canada'
AND ca.category_id=22
AND publish_date_formatted>'2017-11-13';""", con=engine).head()

,video_id,title,published date,trending date,views,likes,country,category_id,category_name
0,qQTVuRrZO8w,Primitive Technology: New area starting from s...,2017-11-24,2017-11-26,4026193,168203,Canada,22,People & Blogs
1,u2Ba65YELoo,The Reaction of The Streets (I Wait-Day6 Edition),2017-12-05,2017-12-06,88889,25599,Canada,22,People & Blogs
2,1Zp_x9BSVVA,lE 21 JANVIER 2018 EYINDI NA KINSHASA,2018-01-21,2018-01-22,77810,311,Canada,22,People & Blogs
3,lddDvegXlRY,Wonder - Julian Gets In Trouble (HD),2018-02-15,2018-02-18,1268346,10565,Canada,22,People & Blogs
4,fbOFogaFF84,"The View March 14, 2018 ; Lena Waithe",2018-03-14,2018-03-15,58534,368,Canada,22,People & Blogs


In [41]:
#####Top Trending video in North America
#- Childish Gambino - This is America (Official Video) was trending in 13 Days in US with view of over 2 M

In [42]:
pd.read_sql_query("""SELECT ca.category_id, ca.category_name, v.video_id, v.title, p.trending_date_formatted as "Trending date",
v.publish_date_formatted as "Published date", p.trending_date_formatted-v.publish_date_formatted as "days to trend", p.views, p.likes,
c.country
FROM videos v
JOIN popularity p
ON v.video_id = p.video_id
JOIN country c
ON p.country_id = c.country_id
JOIN category ca
ON p.category_id = ca.category_id
WHERE p.views>100000000
ORDER BY views DESC;""", con=engine).head()

,category_id,category_name,video_id,title,Trending date,Published date,days to trend,views,likes,country
0,10,Music,VYOjWnS4cMY,Childish Gambino - This Is America (Official V...,2018-06-02,2018-05-06,27,225211923,5023450,United States
1,10,Music,VYOjWnS4cMY,Childish Gambino - This Is America (Official V...,2018-06-01,2018-05-06,26,220490543,4962403,United States
2,10,Music,VYOjWnS4cMY,Childish Gambino - This Is America (Official V...,2018-05-31,2018-05-06,25,217750076,4934188,United States
3,10,Music,VYOjWnS4cMY,Childish Gambino - This Is America (Official V...,2018-05-30,2018-05-06,24,210338856,4836448,United States
4,10,Music,VYOjWnS4cMY,Childish Gambino - This Is America (Official V...,2018-05-29,2018-05-06,23,205643016,4776680,United States
